<a href="https://colab.research.google.com/github/joinbuildclub/buildclub-workshops/blob/main/buildclub_agent_2025-04-17.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install anthropic ipywidgets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.4/243.4 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 20.4 MB/s eta 0:00:00


In [ ]:
import os
import json
import pathlib
from typing import Callable, List, Dict, Any
import ipywidgets as widgets
import anthropic
from google.colab import userdata


client = anthropic.Anthropic(
    api_key=userdata.get('ANTHROPIC_API_KEY'),
)

In [ ]:
class ToolDefinition:
    def __init__(self, name: str, description: str, input_schema: dict, func: Callable[[dict], str]):
        self.name = name
        self.description = description
        self.input_schema = input_schema
        self.func = func

In [ ]:
def read_file(input_data: dict) -> str:
    path = input_data.get("path")
    with open(path, 'r') as f:
        return f.read()

ReadFileDefinition = ToolDefinition(
    name="read_file",
    description="Read the contents of a file.",
    input_schema={"type": "object", "properties": {"path": {"type": "string"}}, "required": ["path"]},
    func=read_file
)

In [ ]:
def list_files(input_data: dict) -> str:
    root = input_data.get("path", ".")
    paths = []
    for dirpath, dirnames, filenames in os.walk(root):
        for name in dirnames + filenames:
            full_path = os.path.join(dirpath, name)
            paths.append(os.path.relpath(full_path, root))
    return json.dumps(paths)

ListFilesDefinition = ToolDefinition(
    name="list_files",
    description="List files and directories at a given path.",
    input_schema={"type": "object", "properties": {"path": {"type": "string"}}, "required": []},
    func=list_files
)

In [ ]:
def create_file(input_data: dict) -> str:
    path = input_data["path"]
    new = input_data["new_str"]

    if not os.path.exists(path):
        os.makedirs(os.path.dirname(path), exist_ok=True)
        with open(path, "w") as f:
            f.write(new)
        return f"Created file {path}"

CreateFileDefinition = ToolDefinition(
    name="create_file",
    description="Create a new file with the contents of `new_str`.",
    input_schema={
        "type": "object",
        "properties": {
            "path": {"type": "string"},
            "new_str": {"type": "string"}
        },
        "required": ["path", "new_str"]
    },
    func=create_file
)

In [ ]:
def edit_file(input_data: dict) -> str:
    path = input_data["path"]
    old = input_data["old_str"]
    new = input_data["new_str"]

    if not os.path.exists(path):
        os.makedirs(os.path.dirname(path), exist_ok=True)
        with open(path, "w") as f:
            f.write(new)
        return f"Created file via edit tool {path}"

    with open(path, "r") as f:
        content = f.read()

    if old not in content:
        raise ValueError("old_str not found in file")

    new_content = content.replace(old, new)

    with open(path, "w") as f:
        f.write(new_content)

    return "OK"

EditFileDefinition = ToolDefinition(
    name="edit_file",
    description="Edit an existing file by replacing `old_str` with `new_str`.",
    input_schema={
        "type": "object",
        "properties": {
            "path": {"type": "string"},
            "old_str": {"type": "string"},
            "new_str": {"type": "string"}
        },
        "required": ["path", "old_str", "new_str"]
    },
    func=edit_file
)

In [ ]:
class Agent:
    def __init__(self, client, get_user_input: Callable[[], str], tools: List[ToolDefinition]):
        self.client = client
        self.get_user_input = get_user_input
        self.tools = tools
        self.conversation = []

    def run(self):
        print("Chat with Claude (press Ctrl+C to exit)")
        while True:
            try:
                user_input = self.get_user_input()
                if not user_input.strip():
                    continue

                self.conversation.append({"role": "user", "content": user_input})

                while True:
                    response = self.infer()
                    self.conversation.append({
                        "role": response["role"],
                        "content": response["content"]
                    })
                    self.pretty_print_claude(response["content"])

                    if response.get("stop_reason") != "tool_use":
                        break  # No more tool calls

                    # Handle tool calls
                    tool_result_messages = []
                    for call in response["content"]:
                        if call["type"] == "tool_use":
                            print(f"\033[92mClaude wants to use tool\033[0m: {call['name']} with input: {json.dumps(call['input'], indent=2)}")
                            result = self.handle_tool_call(call)

                            tool_result_messages.append({
                                "role": "user",
                                "content": [{
                                    "type": "tool_result",
                                    "tool_use_id": call["id"],
                                    "content": result
                                }]
                            })

                            print(f"\033[95mTool result\033[0m: {result}\n")

                    self.conversation.extend(tool_result_messages)

            except KeyboardInterrupt:
                break

    def pretty_print_claude(self, content):
        for block in content:
            if block["type"] == "text":
                print(f"\033[93mClaude\033[0m: {block['text']}\n")
            elif block["type"] == "tool_use":
                print(f"\033[92mClaude wants to use tool\033[0m: {block['name']} with input: {json.dumps(block['input'], indent=2)}\n")
            else:
                print(f"\033[90m[Unknown block type]\033[0m: {block}")

    def infer(self) -> dict:
        tools_param = [{
            "name": tool.name,
            "description": tool.description,
            "input_schema": tool.input_schema
        } for tool in self.tools]

        response = self.client.messages.create(
            model="claude-3-7-sonnet-latest",
            max_tokens=1024,
            messages=self.conversation,
            tools=tools_param
        )
        return response.model_dump()

    def handle_tool_call(self, call: dict) -> str:
        tool = next((t for t in self.tools if t.name == call["name"]), None)
        if not tool:
            return f"Tool {call['name']} not found"
        result = tool.func(call["input"])
        return result

In [ ]:
def get_input():
    return input("\033[1mYou:\033[0m ")

tools = [ReadFileDefinition, ListFilesDefinition, CreateFileDefinition, EditFileDefinition]
agent = Agent(client, get_input, tools)
agent.run()

Chat with Claude (press Ctrl+C to exit)
You: what files are in this dir
Claude: I'll check what files are in the current directory for you.

Claude wants to use tool: list_files with input: {}

Claude wants to use tool: list_files with input: {}
Tool result: [".config", "sample_data", ".config/configurations", ".config/logs", ".config/default_configs.db", ".config/.last_update_check.json", ".config/.last_survey_prompt.yaml", ".config/config_sentinel", ".config/gce", ".config/.last_opt_in_prompt.yaml", ".config/hidden_gcloud_config_universe_descriptor_data_cache_configs.db", ".config/active_config", ".config/configurations/config_default", ".config/logs/2025.04.11", ".config/logs/2025.04.11/13.36.51.456109.log", ".config/logs/2025.04.11/13.36.21.789230.log", ".config/logs/2025.04.11/13.37.01.850798.log", ".config/logs/2025.04.11/13.37.02.623021.log", ".config/logs/2025.04.11/13.36.42.509045.log", ".config/logs/2025.04.11/13.36.52.690687.log", "sample_data/README.md", "sample_data/anscom